In [84]:
import import_ipynb
import numpy as np
import glob
import matplotlib.pyplot as plt
import datetime

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [85]:
batch_size = 32

## Model callbacks

In [86]:
tb_log_dir = "../log/tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
cp_filepath = '../log/cps/{model.name}_cp_{epoch}.h5'

callbacks = [
    keras.callbacks.ModelCheckpoint(                        
                        filepath=cp_filepath, 
                        verbose=1, 
                        save_weights_only=False,
                        save_freq=5*batch_size), # Checkpoint is saved every five epochs
                        tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1)
]

## Data parameters

In [87]:
image_height = 224
image_width = 224

## Creating the training dataset

In [88]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 84 files belonging to 2 classes.
Using 68 files for training.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Creating the validation dataset

In [89]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

Found 84 files belonging to 2 classes.
Using 16 files for validation.


## Creating the test dataset

In [90]:
test_ds = []

## Load the base model

In [91]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
base_model = ResNet50(
                include_top=False,
                weights='imagenet',  # Load weights pre-trained on ImageNet.
                input_shape=(image_height, image_width, 3)
                )
base_model.trainable = False

## Creating a new model

In [92]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Running base model in inference mode
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

## Training the new model with new data

In [93]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
            face_training_ds, 
            epochs=10, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

Epoch 1/10
3/3 [==============================] - 23s 6s/step - loss: 1.0133 - accuracy: 0.2353 - val_loss: 0.5867 - val_accuracy: 0.5625
Epoch 2/10
3/3 [==============================] - 12s 5s/step - loss: 0.5822 - accuracy: 0.6029 - val_loss: 0.3500 - val_accuracy: 0.8125
Epoch 3/10
3/3 [==============================] - 13s 5s/step - loss: 0.3546 - accuracy: 0.8971 - val_loss: 0.2242 - val_accuracy: 0.9375
Epoch 4/10
3/3 [==============================] - 13s 5s/step - loss: 0.1949 - accuracy: 1.0000 - val_loss: 0.1532 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 13s 5s/step - loss: 0.1192 - accuracy: 1.0000 - val_loss: 0.1165 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 12s 5s/step - loss: 0.0808 - accuracy: 1.0000 - val_loss: 0.0930 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 12s 5s/step - loss: 0.0590 - accuracy: 1.0000 - val_loss: 0.0756 - val_accuracy: 1.0000
Epoch 8/10
3/3 [==================

## View Results

## Predict new images

In [94]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/FaceTest/000001.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)
score = preds[0][0]

if(score > 0.5):
    print("This image contains no face. ({:.2f} percent confidence)".format(100 * score))
else:
    print("This image contains a face. ({:.2f} percent confidence)".format(100 * (1-score)))

1/1 [==============================] - 2s 2s/step
This image contains a face. (97.26 percent confidence)
